## 目标：从串口接收传感器波形

监护仪终端设备的心电波形数据是从心电采集模块传过来的，而心电采集模块是一个独立的嵌入式模块，简单点说是个单片机，心电模块与主控(运行Qt界面的CPU)之间通过串口进行数据通信。

这个串口是相对于并口来讲的，并口就像马路上并排行驶的汽车，而串口就像前方到道路变窄，每次只能通过一辆车。

本以为领导会给我一个心电采集模块进行调试的。结果领导要求我自己找个虚拟串口，然后写2个qt程序，两者实现数据互传。

## 环境准备

### 虚拟串口

https://freevirtualserialports.com/  
或者  
链接：https://pan.baidu.com/s/1m7d1I9-zcJMxDe4sb_VNrA   
提取码：s7wg

### 串口助手
串口助手是可以从串口接收数据或给串口发送数据的小工具，串口通信调试经常需要。  
链接：https://pan.baidu.com/s/1ePFPnMxAbguoxRvTmlc5aA   
提取码：qku8

工具使用如下,虚拟串口工具创建了1个桥接的串口COM4,COM5，2个串口助手分别给对方发送一个字符串，并分别收到对方发送的数据
![](./image/qt_serial_tools.png)

需要注意的是串口助手发送数据的格式有2中十六进制发送和字符串发送

# Qt的QSerial和QSerialPortInfo

Qt提供了2个与串口相关的模块：QSerial和QSerialPortInfo。  

QSerial：用于访问串口，如打开串口、配置串口的参数（如波特率、停止位、奇偶校验位等）、同时提供数据的传输等
QSerialPortInfo：用于获取当前设备的串口信息，例如有多少个串口、串口号之类的信息

## 查看本机串口信息

假如你已经启动了虚拟串口工具，并且创建了2个串口：COM4和COM5，COM后面的数字是串口号和自行选择。

下面让我们导入模块，并查询一下串口信息，Qt提供了很好的接口函数，我们可以通过availablePorts查看当前设备有哪些可用串口：

In [ ]:
from PyQt5.QtSerialPort import QSerialPortInfo

serialPortList = QSerialPortInfo.availablePorts()
print(len(serialPortList))

`availablePorts`返回了一个串口信息对象列表，我们可以查看端口名称`portName`和端口号`serialNumber`

In [ ]:
for index in range(len(serialPortList)):
    print("[{:d}]".format(index), serialPortList[index].portName(),";", serialPortList[index].serialNumber(),";", serialPortList[index].description())

虽然QSerialPortInfo还提供了很多关于串口设备的信息，例如厂商、产品识别码等，但是一般工程上最重要的还是串口号即`portName`。并且我们可以发现上面并没有打印出`seruialNumber`

In [ ]:
print(serialPortList[0].serialNumber() is None)

serialNumber的问题以后再做研究，下面让我们打开串口4。

## 串口数据接收

串口的使用需要：
1. 创建串口对象
2. 配置端口号、波特率等参数
3. 打开串口
4. 读写串口

首先是创建串口对象：

In [ ]:
from PyQt5.QtSerialPort import QSerialPort

serialPort = QSerialPort()
serialPort.setPortName('COM4')

我们创建了一个串口对象，并配置为COM4，接下来就可以打开这个串口，并做相关参数的配置。例如串口的波特率、数据位、校验方式、流控方式、停止位等参数。

In [ ]:
if serialPort.open(QSerialPort.ReadWrite) is False:
    print("open serial port false")
else:
    print("open serial port done")
    
serialPort.setBaudRate(QSerialPort.Baud115200) 
serialPort.setDataBits(QSerialPort.Data8)
serialPort.setStopBits(QSerialPort.OneStop)
serialPort.setParity(QSerialPort.NoParity)
serialPort.setFlowControl(QSerialPort.NoFlowControl)

波特率配置为115200，数据位为8，1个停止位，无校验，无流控。串口的打开和配置就完成了。让我们通过串口助手发个数据过来，看是否能正常接收。这里我们做一个死循环进行数据的不断接收并打印。

In [ ]:
while True:
    revData = serialPort.readAll()
    if revData.size() is not 0:
        print(revData)

执行上面代码我们发现并不能接收到任何数据。

原因还是Qt事件机制，串口的数据收发会触发一些事件，但是没有启动QApplication的情况下没有事件的调度处理。但是如果执行了app.exec_()后就没办法监听串口接收了。这里我们用定时器+信号与槽机制来解决这个问题。

## 创建串口接口类

为此，需要定义串口类,在类的初始化中完成串口的打开和参数配置，同时在初始化创建定时器并配置槽函数为读数据函数。
### SerialInterface_V0.1

In [ ]:
from PyQt5.QtWidgets import QApplication
from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtCore import pyqtSignal, QObject, QTimer

class SerialInterface(QObject):
    def __init__(self,port='COM4'):
        super().__init__()

        self.com = QSerialPort()
        
        self.com.setPortName(port) 
        self.com.setBaudRate(QSerialPort.Baud115200) 
        self.com.setDataBits(QSerialPort.Data8)
        self.com.setStopBits(QSerialPort.OneStop)
        self.com.setParity(QSerialPort.NoParity)
        self.com.setFlowControl(QSerialPort.NoFlowControl)
        
        if self.com.open(QSerialPort.ReadWrite) == False:
            print("serial port open error")
            return

        self.readtimer = QTimer()
        self.readtimer.timeout.connect(self.readData)
        self.readtimer.start(100) # 100ms周期定时器
 
    def readData(self):
        revData = self.com.readAll()
        if revData.size() is not 0:
            print(revData)


In [ ]:
app = QApplication([])

serial = SerialInterface('COM4')

app.exec_()

还不错，我们成功接收到了串口数据。接下来要面临的有2个问题：

第一个问题是，串口会接收到很多数据，如何从里面知道哪些是心电波形数据，即便是所有数据都是心电电压数据，那么心电数据是16位的，如何知道哪个是搞字节，哪个是低字节。为解决这个问题需要引入通信协议的概念

第二个问题是，串口接收到的数据如何交给波形显示模块。如何解决模块之间的数据交互。

# 通信协议

## 通信协议介绍

通信协议有点像对暗号，就像谍战中常见的剧情“敲门”，“咚咚--咚--咚”表示是我请开门，“咚--咚咚--咚”表示赶紧撤离。还有个剧情就是“天王盖地虎”“小鸡炖蘑菇”。

这里都有个共同点，双方都不知道对方是谁，需要通过一个特定的信号来进行认证。串口通信也是如此，我们在数据流中加入一些特定的符号来表示有效数据的起始或结束。例如：所有的数据内容都是由ascii组成，数据中不包含回车，收到回车表示数据发送完成，每次发送会有多个命令，不同命令之间用';'隔离。这是一个极简单的通信协议了，但是这里使用的是字符串格式的数据传输。

所以通信协议本质上是对数据内容的封装或标记，主要包含以下内容：
起始标志 + 数据长度 + 数据内容 + 校验结果 + 结束标志

心电数据是十六进制的，十六进制与字符串的区别是，数据内容可以是任何数字，所以就有可能数据内容中存在起始标志的情况发送，这样很容易造成数据传输的错误，因为接收端不能知道正确数据长度进而不能得到正确的验证结果，所以总是会认为接收到的数据是错误的。例如起始标准为 0x05, 而数据内容中包含有设备的ID，设备ID包含有0x05,这样如何区分接收到的0x55是起始标志还是设备ID呢。

解决这个问题有不同的实现方法：
* 方法1：使用组合字符避免出现0x5, 例如定义0x00~0x0F之间的字符为特殊含义，例如用于起始标志、结束标志等，在发送端遇到0~0x0F之间的数据时，在数据上增加0x10，使数据变为0x10~0x1F，为了标志数据已发生改变在这个数据前加上一个数据0x02,所以0x03--> 0x02 0x13, 0x02-->0x02 0x12。接收端根据同样的原则进行数据解析即可。
* 方法2：在起始标志和数据长度后面紧跟一个校验结果，用于校验起始标志和数据长度字段是否正确，如果正确继续接收剩余数据，如果不正确跳过当前起始标志，从下一个字节开始解析
* 其他方法

心电数据采用的是哪种方法呢，至少我们使用的模块采用方案是其他方案...不过我们已经知道了通信协议的本质和他们要解决的问题。那么我们来分析下心电的数据协议吧：

|字段|内容/含义|
|:--:|:--:|
|ID|0x08|
|数据|数据头|
|数据|ECG1 波形数据高字节|
|数据|ECG1 波形数据低字节|
|数据|ECG2 波形数据高字节|
|数据|ECG2 波形数据低字节|
|数据|ECG3 波形数据高字节|
|数据|ECG3 波形数据低字节|
|数据|ECG 状态Status|
|校验和||

提示：ID不一定与真实产品一致
因为此数据内容固定，所以没有数据长度字段，而是根据约定的ID对应的字段内容知道数据长度的。具体示例如下

数据示例：8 a8 8a a9 87 cb 85 84 80 bb 

可以发现除ID外后面的数据部分和校验部分的最高位都为1，这个正是协议的特殊之处，数据头部分字段的低7位存放的是后面7个字节的最高位，同时校验和取的也是数据字段的低7位的结果。
如下图解析：
![](./image/serial_ecg_protocols.png)


## 通信协议数据解析

首先我们使用上面的串口接收程序把收到的心电数据打印出来，数据是用串口助手用十六进制模式发出来的。

In [ ]:
# 1. 执行 SerialInterface_V0.1
# 2. 当前代码段
 
from PyQt5.QtWidgets import QApplication
from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtCore import pyqtSignal, QObject, QTimer

app = QApplication([])

serial = SerialInterface('COM4')

app.exec_()

看上去数据接收良好。这是因为我们只发了一个数据，并且是手动发送的。而实际情况是每秒中会有500个这样的是数据，这样而我们需要每2ms就发送一次。修改串口助手循环发送，发送间隔为2ms进行测试。上面代码输出的日志如下：
```
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
b'\x08\xa8\x8a\xa9\x87\xcb\x85\x84\x80\xbb'
```

可以看出读取数据的时候不是一条一条的，有可能一次读出多条数据，日志中虽然没有出现，但是我们可以猜测会有可能读数的时候数据还没有发完，这样可能只读到一半的数据。这种现象在串口通信中是非常常见的。

这里给出的处理思路是：
* 每次从串口读取一个数据
* 然后定义一个带数据缓存功能的协议解析函数
* 解析函数中完成ID的识别，数据内容的接收解析，还原出原始数据，并发出信号用于更新波形

暂时我们先尝试还原出原始数据并进行打印。这里直接定义了一个类来实现数据的解析

### EcgDecode_0v1

In [ ]:
class EcgDecode():
    def __init__(self):
        self.status = 0 # 0-idle, 1-rxId, 2-rxDataHead, 3-RxDone
        self.head = 0
        self.dataCnt = 0
        self.crc = 0
        self.data = []
        self.ecgData = []
        print("ecgDecode init done")
        
    def dataHandle(self,data):
        if data < 0x80:
            if data == 0x08:
                self.status = 1
                self.dataCnt = 0
                self.data.clear()
                return self.status
            else:
                print("unknown id", data)
                self.status = 0
                return self.status

        if self.status == 1:
            self.head = data
            self.status = 2
            return self.status

        if self.status == 2:
            # 数据解析，并从数据头中获取数据的最高位
            tmpData = data
            tmpData = tmpData & 0x7F
            tmpData = tmpData | (((self.head >> self.dataCnt) & 0x01)<<7)
            # 数据缓存并计数
            self.data.append(tmpData)
            self.dataCnt += 1
            if self.dataCnt >= 7:
                self.status = 3
            return self.status
        
        if self.status == 3:
            if self.crcCheck(data) == True:
                self.status = 4 # rx done
                print("rx package done")
                
                self.ecgData.clear()
                self.ecgData.append(self.data[0]*0x100 + self.data[1])
                self.ecgData.append(self.data[2]*0x100 + self.data[3])
                self.ecgData.append(self.data[4]*0x100 + self.data[5])
                # 数据打印
                print("ecg1=",self.ecgData[0])
                print("ecg2=",self.ecgData[1])
                print("ecg3=",self.ecgData[2])
            else:
                self.status = 0
                print("rx package crc error")
            return self.status
    
    def crcCheck(self, crc):
        # toDo crc check
        return True

对串口接收代码略加修改。
* self.ecgDecode = EcgDecode() 创建数据解析对象
* self.ecgDecode.dataHandle(revData[0]) 接收到数据后用数据处理函数完成数据的解析缓存

### SerialInterface_0v2

In [ ]:
from PyQt5.QtCore import QObject
class SerialInterface(QObject):
    def __init__(self,port='COM4'):
        super().__init__()

        self.com = QSerialPort()
        
        self.com.setPortName(port) 
        self.com.setBaudRate(QSerialPort.Baud115200) 
        self.com.setDataBits(QSerialPort.Data8)
        self.com.setStopBits(QSerialPort.OneStop)
        self.com.setParity(QSerialPort.NoParity)
        self.com.setFlowControl(QSerialPort.NoFlowControl)
        
        if self.com.open(QSerialPort.ReadWrite) == False:
            print("serial port open error")
            return

        self.readtimer = QTimer()
        self.readtimer.timeout.connect(self.readData)
        self.readtimer.start(100) # 100ms周期定时器
        
        self.ecgDecode = EcgDecode()
        print("serial ready")
 
    def readData(self):
        revData = self.com.read(1)
        if len(revData) > 0:
            self.ecgDecode.dataHandle(revData[0])


In [ ]:
# 1. SerialInterface_0v2
# 2. ecgDecode_0v1
from PyQt5.QtWidgets import QApplication
from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtCore import pyqtSignal, QObject, QTimer
            
app = QApplication([])

serial = SerialInterface('COM4')

app.exec_()

程序启动后在串口终端分别发送数据：
```
8 80 88 80 88 80 88 80 80 9d 
8 8a 87 a3 87 80 87 ee 81 b6
```
然后可以得到日志如下：
```
ecgDecode init done
serial ready
rx package done
ecg1= 2048
ecg2= 2048
ecg3= 2048
rx package done
ecg1= 1955
ecg2= 1920
ecg3= 1902
```
可见数据解析正确。校验字段的使用需根据发送端所使用的校验方式进行检验。

现在已经有数据了，怎么把数据显示到界面上呢？

信号与槽

不错，信号与槽机制应该是个不错的选择，当收到数据的时候发出信号给界面的绘图函数，使界面把新收到的数据显示在界面上。

# 串口数据显示到波形界面

### EcgDecode_0v2

删除部分日志

In [2]:
class EcgDecode():
    def __init__(self):
        self.status = 0 # 0-idle, 1-rxId, 2-rxDataHead, 3-RxDone
        self.head = 0
        self.dataCnt = 0
        self.crc = 0
        self.data = []
        self.ecgData = []
        print("ecgDecode init done")
        
    def dataHandle(self,data):
        if data < 0x80:
            if data == 0x08:
                self.status = 1
                self.dataCnt = 0
                self.data.clear()
                return self.status
            else:
                print("unknown id", data)
                self.status = 0
                return self.status

        if self.status == 1:
            self.head = data
            self.status = 2
            return self.status

        if self.status == 2:
            # 数据解析，并从数据头中获取数据的最高位
            tmpData = data
            tmpData = tmpData & 0x7F
            tmpData = tmpData | (((self.head >> self.dataCnt) & 0x01)<<7)
            # 数据缓存并计数
            self.data.append(tmpData)
            self.dataCnt += 1
            if self.dataCnt >= 7:
                self.status = 3
            return self.status
        
        if self.status == 3:
            if self.crcCheck(data) == True:
                self.status = 4 # rx done
#                 print("rx package done")
                
                self.ecgData.clear()
                self.ecgData.append(self.data[0]*0x100 + self.data[1])
                self.ecgData.append(self.data[2]*0x100 + self.data[3])
                self.ecgData.append(self.data[4]*0x100 + self.data[5])
                # 数据打印
#                 print("ecg1=",self.ecgData[0])
#                 print("ecg2=",self.ecgData[1])
#                 print("ecg3=",self.ecgData[2])
            else:
                self.status = 0
                print("rx package crc error")
            return self.status
    
    def crcCheck(self, crc):
        # toDo crc check
        return True

## 串口接口类中定义信号

此类中定义信号和发信号函数，在串口数据接收中根据解码对象的返回状态判断是否完成数据接收，如果完成则发送信号

### SerialInterface_0v3

修改定时器周期为 10ms

In [ ]:
from PyQt5.QtCore import pyqtSignal, QObject
class SerialInterface(QObject):
    signal = pyqtSignal(int) # 定义信号，pyqt5信号要定义为类属性，而不是放在 _init_这个方法里面
    def __init__(self,port='COM4'):
        super().__init__()

        self.com = QSerialPort()
        
        self.com.setPortName(port) 
        self.com.setBaudRate(QSerialPort.Baud115200) 
        self.com.setDataBits(QSerialPort.Data8)
        self.com.setStopBits(QSerialPort.OneStop)
        self.com.setParity(QSerialPort.NoParity)
        self.com.setFlowControl(QSerialPort.NoFlowControl)
        
        if self.com.open(QSerialPort.ReadWrite) == False:
            print("serial port open error")
            return

        self.readtimer = QTimer()
        self.readtimer.timeout.connect(self.readData)
        self.readtimer.start(10) # 10ms周期定时器
        
        self.ecgDecode = EcgDecode()
        print("serial ready")
    
    def sendSignal(self, data):
#         print("send signal with data", data)
        self.signal.emit(data)
    
    def readData(self):
        revData = self.com.read(1)
        if len(revData) > 0:
            if 4 == self.ecgDecode.dataHandle(revData[0]):
                self.sendSignal(self.ecgDecode.ecgData[1])

In [ ]:
# 1. 串口数据接收类 SerialInterface_0v3
# 2. 串口数据解析类 EcgDecode_0v2
# 3. 当前代码段

from PyQt5.QtWidgets import QApplication
from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtCore import pyqtSignal, QObject, QTimer

app = QApplication([])

serial = SerialInterface('COM4')

app.exec_()

### QWidgetDraw_0v1

波形的显示界面函数如下：
* 修改定时器处理函数，使默认绘制直线波形，当串口没有及时收到数据时会绘制直线
* 当串口发出信号时，在串口信号处理函数中重启定时器，把收到的数据绘制到界面
* 波形绘制部分删除了原来的demo数据内容，波形数据从信号槽传送到界面

需要思考并解决的问题：
* 如何实现波形数据的抽样
* 多个串口同时接收不同的波形数据该如何处理

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget

class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(100, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.lastData = 0
    
    def timerEvent(self, event):
        self.drawWaveToPixmap(2048)
        self.update()
        
    # 串口数据信号槽函数
    def slotSerialDataHandle(self, data):
        self.drawWaveToPixmap(data)
        self.timer.start(100, self)
        self.update()
        
    def drawWaveToPixmap(self, data=None):
        if self.lastData == 0:
            self.lastData = data
            return
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter, data) # 把接收到的数据绘制到pixmap上
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
        
    def drawEcgWave(self, painter, data):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)
        
        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        pen.setWidth(8)
        painter.drawLine(self.index+4, 0, self.index+4, self.height())
        painter.restore() # 还原当前配置
        
        # 计算绘图参数
        height = self.height()

        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 2
        if self.index > self.width():
            self.index = 2

        lineStart = QPoint()
        lineEnd = QPoint()
        
        # 设置起始、结束点的x坐标
        lineStart.setX(self.index-2)
        lineEnd.setX(self.index)
        # 设置起始点的y坐标
        lineStart.setY(round(self.height()/2 - ((self.lastData - 2048)/600) * self.height()/2))
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((data - 2048)/600) * self.height()/2))
        
        # 直线绘制
        painter.drawLine(lineStart, lineEnd)
                        
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)

窗口界面类中，定义了串口信号的槽函数：
```
    # 串口数据信号槽函数
    def slotSerialDataHandle(self, data):
        self.drawWaveToPixmap(data)
        self.timer.start(30, self)
        self.update()
```

同时因为传进来的有数据参数，所以需要修改类中相关函数。其中也发现一个bug就是，原来代码中：
```
        painter.setPen(pen)
        painter.drawLine(self.index, 0, self.index+4, self.height())
```
应变更为:
```
        painter.setPen(pen)
        pen.setWidth(8)
        painter.drawLine(self.index+4, 0, self.index+4, self.height())
```
当然这里也可以绘制一个矩形，这里用于擦除之前绘制的局部波形，用于绘制新波形的，但是实际上绘制的是一条斜线。

In [ ]:
## 先执行
# 1. 串口数据接收类 SerialInterface_0v3
# 2. 串口数据解析类 EcgDecode_0v1
# 3. 界面显示类 QWidgetDraw_0v1

import math

from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import pyqtSignal, QObject, QTimer, Qt, QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

serial = SerialInterface('COM4')

wave = EcgWave()
wave.resize(600, 150)

serial.signal.connect(wave.waveWin.slotSerialDataHandle)

wave.show()

app.exec_()

![](./image/qt_serial_wave.png)

从上面波形可以看出一些问题：
1. 串口全速发送数据时，输出波形实际上是个锯齿，这说明串口数据接收、解析的速度有点慢。可是为什么呢

当串口停止发数后，有较长时间段内仍然是锯齿状波形，说明数据有很多在等待处理。

我们分析串口接收处理函数发现，实际上代码是有问题的，串口数据的接收是定时器访问的，但是每次只取了一个数据，那么取一组数据的时间就是10*定时器周期了。所以最好的办法时串口数据处理时把所有数据处理完再退出。

### SerialInterface_0v4

In [3]:
from PyQt5.QtCore import pyqtSignal, QObject
class SerialInterface(QObject):
    signal = pyqtSignal(int) # 定义信号，pyqt5信号要定义为类属性，而不是放在 _init_这个方法里面
    def __init__(self,port='COM4'):
        super().__init__()

        self.com = QSerialPort()
        
        self.com.setPortName(port) 
        self.com.setBaudRate(QSerialPort.Baud115200) 
        self.com.setDataBits(QSerialPort.Data8)
        self.com.setStopBits(QSerialPort.OneStop)
        self.com.setParity(QSerialPort.NoParity)
        self.com.setFlowControl(QSerialPort.NoFlowControl)
        
        if self.com.open(QSerialPort.ReadWrite) == False:
            print("serial port open error")
            return

        self.readtimer = QTimer()
        self.readtimer.timeout.connect(self.readData)
        self.readtimer.start(1) # 1ms周期定时器
        
        self.ecgDecode = EcgDecode()
        print("serial ready")
    
    def sendSignal(self, data):
#         print("send signal with data", data)
        self.signal.emit(data)
    
    def readData(self):
        while self.com.atEnd() is  False:
            revData = self.com.read(1)
            if len(revData) > 0:
                if 4 == self.ecgDecode.dataHandle(revData[0]):
                    self.sendSignal(self.ecgDecode.ecgData[1])
            else:
                return

In [ ]:
## 先执行
# 1. 串口数据接收类 SerialInterface_0v4
# 2. 串口数据解析类 EcgDecode_0v1
# 3. 界面显示类 QWidgetDraw_0v1

import math

from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import pyqtSignal, QObject, QTimer, Qt, QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

serial = SerialInterface('COM4')

wave = EcgWave()
wave.resize(600, 150)

serial.signal.connect(wave.waveWin.slotSerialDataHandle)

wave.show()

app.exec_()

![](./image/qt_serial_wave_2.png)
波形有所改善，但是可以看到还是有很大问题，好像定时器绘图和串口数据绘图在交替执行，这里把`QWidgetDraw`中定时器周期调大到100ms进行简单测试。得到如下波形：
![](./image/qt_serial_wave_3.png)
从这个波形看上去比之前更好点，想到绘图的点实际上是有问题的，程序中没有更新`lastPoint`

### QWidgetDraw_0v2 bug

修改代码段，增加`self.lastData = data`:
```
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((data - 2048)/600) * self.height()/2))
        self.lastData = data
```

In [3]:
from PyQt5.QtWidgets import QApplication, QWidget

class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(100, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.lastData = 0
    
    def timerEvent(self, event):
        if event.timerId() == self.timer.timerId():
            self.drawWaveToPixmap(2048)
            self.update()
        else:
            print("skip unknown timerId event:", event.timerId)
        
    # 串口数据信号槽函数
    def slotSerialDataHandle(self, data):
        self.drawWaveToPixmap(data)
        self.timer.stop()
        self.timer.start(100, self)
        self.update()
        
    def drawWaveToPixmap(self, data=None):
        if self.lastData == 0:
            self.lastData = data
            return
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter, data) # 把接收到的数据绘制到pixmap上
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
        
    def drawEcgWave(self, painter, data):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)
        
        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        pen.setWidth(8)
        painter.drawLine(self.index+4, 0, self.index+4, self.height())
        painter.restore() # 还原当前配置
        
        # 计算绘图参数
        height = self.height()

        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 2
        if self.index > self.width():
            self.index = 2

        lineStart = QPoint()
        lineEnd = QPoint()
        
        # 设置起始、结束点的x坐标
        lineStart.setX(self.index-2)
        lineEnd.setX(self.index)
        # 设置起始点的y坐标
        lineStart.setY(round(self.height()/2 - ((self.lastData - 2048)/600) * self.height()/2))
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((data - 2048)/600) * self.height()/2))
        self.lastData = data
        
        # 直线绘制
        painter.drawLine(lineStart, lineEnd)
                        
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)

In [ ]:
## 先执行
# 1. 串口数据接收类 SerialInterface_0v4
# 2. 串口数据解析类 EcgDecode_0v1
# 3. 界面显示类 QWidgetDraw_0v2

import math

from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import pyqtSignal, QObject, QTimer, Qt, QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

serial = SerialInterface('COM4')

wave = EcgWave()
wave.resize(600, 150)

serial.signal.connect(wave.waveWin.slotSerialDataHandle)

wave.show()

app.exec_()

ecgDecode init done
serial ready


![](./image/qt_serial_wave_4.png)

更简单的方案是：
停止定时器绘图



-------------------------------------------------------------
**此部分内容暂时不用**
~~! [](./image/qt_serial_wave_4.png)~~

看上去问题很大程度上得到了解决。中间的尖尖很像是定时器绘图的结果，增加日志进行观察：
```
    def timerEvent(self, event):
        self.drawWaveToPixmap(2048)
        self.update()
        print("timerEvent")
```
执行程序发现确实会不停的产生定时器事件。虽然数据手册上有说明start函数可以启动或重启定时器。单实际上并没有重启,所以我们可以认为这里`start`只是定时器的启动,入口参数可以重新配置定时周期，但是不能让当前计数值清零

这么看来，同时用定时和串口数据处理槽函数进行绘图就不具备可行性了。进一步分析我们可以想到只让串口处理函数提供数据，数据绘图仍然有定时器来做，这样还有个好处是可以控制波形的刷新速度。所以在槽函数`slotSerialDataHandle`中完成的事情就是把数据存入缓冲区，定时器时间处理函数`timerEvent`中要做的事情是取出缓冲区中最早存入的数据并绘制到界面上。

这里实际上引入了概念**FIFO**，数字电路也有这个逻辑电路，先入先出。就是最先存入一个缓冲区的数据将最先被调用。qt为我们提供了一个满足此需求的控件-**Queue**（队列），队列有2个主要的函数是`enqueue`入列和`dequeue`出列，前者提供了存储缓存区的方法，后者从缓冲区取出最先存入的数据。

修改slotSerialDataHandle函数如下：
```
    # 串口数据信号槽函数
    def slotSerialDataHandle(self, data):
        self.queue.enqueue(data)
```

修改timerEvent函数如下：
```
    def timerEvent(self, event):
        if event.timerId() == self.timer.timerId():
            if self.queue.isEmpty():
                self.drawWaveToPixmap(2048)
            else
                self.drawWaveToPixmap(self.queue.dequeue())
            self.update()
        else:
            print("skip unknown timerId event:", event.timerId)
```
同时需要在初始化函数中定义queue

### ~~QWidgetDraw_0v3 Queue~~

然而遗憾的是pyqt5中没有支持QQueue，因为qt的QQueue需要用到C++的一些特性，所以只在C++版本支持。代码中使用的是python的queue类。

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget
import queue

class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        self.queue = queue.Queue()
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.lastData = 0
    
    def timerEvent(self, event):
        if event.timerId() == self.timer.timerId():
            if self.queue.empty():
                self.drawWaveToPixmap(2048)
            else:
                self.drawWaveToPixmap(self.queue.get())
            self.update()
        else:
            print("skip unknown timerId event:", event.timerId)
            
    # 串口数据信号槽函数
    def slotSerialDataHandle(self, data):
        self.queue.put(data)
        
    def drawWaveToPixmap(self, data=None):
        if self.lastData == 0:
            self.lastData = data
            return
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter, data) # 把接收到的数据绘制到pixmap上
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
        
    def drawEcgWave(self, painter, data):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)
        
        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        pen.setWidth(8)
        painter.drawLine(self.index+4, 0, self.index+4, self.height())
        painter.restore() # 还原当前配置
        
        # 计算绘图参数
        height = self.height()

        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 2
        if self.index > self.width():
            self.index = 2

        lineStart = QPoint()
        lineEnd = QPoint()
        
        # 设置起始、结束点的x坐标
        lineStart.setX(self.index-2)
        lineEnd.setX(self.index)
        # 设置起始点的y坐标
        lineStart.setY(round(self.height()/2 - ((self.lastData - 2048)/600) * self.height()/2))
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((data - 2048)/600) * self.height()/2))
        self.lastData = data
        
        # 直线绘制
        painter.drawLine(lineStart, lineEnd)
                        
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)

In [ ]:
## 先执行
# 1. 串口数据接收类 SerialInterface_0v4
# 2. 串口数据解析类 EcgDecode_0v1
# 3. 界面显示类 QWidgetDraw_0v2

import math

from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import pyqtSignal, QObject, QTimer, Qt, QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

serial = SerialInterface('COM4')

wave = EcgWave()
wave.resize(600, 150)

serial.signal.connect(wave.waveWin.slotSerialDataHandle)

wave.show()

app.exec_()

# 串口数据波形显示

把相关代码整合如下：


In [1]:
class EcgDecode():
    def __init__(self):
        self.status = 0 # 0-idle, 1-rxId, 2-rxDataHead, 3-RxDone
        self.head = 0
        self.dataCnt = 0
        self.crc = 0
        self.data = []
        self.ecgData = []
        print("ecgDecode init done")
        
    def dataHandle(self,data):
        if data < 0x80:
            if data == 0x05:
                self.status = 1
                self.dataCnt = 0
                self.data.clear()
                return self.status
            else:
                print("unknown id", data)
                self.status = 0
                return self.status

        if self.status == 1:
            self.head = data
            self.status = 2
            return self.status

        if self.status == 2:
            # 数据解析，并从数据头中获取数据的最高位
            tmpData = data
            tmpData = tmpData & 0x7F
            tmpData = tmpData | (((self.head >> self.dataCnt) & 0x01)<<7)
            # 数据缓存并计数
            self.data.append(tmpData)
            self.dataCnt += 1
            if self.dataCnt >= 7:
                self.status = 3
            return self.status
        
        if self.status == 3:
            if self.crcCheck(data) == True:
                self.status = 4 # rx done
                
                self.ecgData.clear()
                self.ecgData.append(self.data[0]*0x100 + self.data[1])
                self.ecgData.append(self.data[2]*0x100 + self.data[3])
                self.ecgData.append(self.data[4]*0x100 + self.data[5])
            else:
                self.status = 0
                print("rx package crc error")
            return self.status
    
    def crcCheck(self, crc):
        # toDo crc check
        return True

In [2]:
from PyQt5.QtCore import pyqtSignal, QObject
class SerialInterface(QObject):
    signal = pyqtSignal(int) # 定义信号，pyqt5信号要定义为类属性，而不是放在 _init_这个方法里面
    def __init__(self, port = 'COM4'):
        super().__init__()

        self.com = QSerialPort()
        
        self.com.setPortName(port) 
        self.com.setBaudRate(QSerialPort.Baud115200) 
        self.com.setDataBits(QSerialPort.Data8)
        self.com.setStopBits(QSerialPort.OneStop)
        self.com.setParity(QSerialPort.NoParity)
        self.com.setFlowControl(QSerialPort.NoFlowControl)
        
        if self.com.open(QSerialPort.ReadWrite) == False:
            print("serial port open error")
            return

        self.readtimer = QTimer()
        self.readtimer.timeout.connect(self.readData)
        self.readtimer.start(1) # 1ms周期定时器
        
        self.ecgDecode = EcgDecode()
        print("serial ready")
    
    def sendSignal(self, data):
#         print("send signal with data", data)
        self.signal.emit(data)
    
    def readData(self):
        while self.com.atEnd() is  False:
            revData = self.com.read(1)
            if len(revData) > 0:
                if 4 == self.ecgDecode.dataHandle(revData[0]):
                    self.sendSignal(self.ecgDecode.ecgData[1])
            else:
                return

In [3]:
## 此部分代码应使用QwidgetDraw_0v2 bug版本

from PyQt5.QtWidgets import QApplication, QWidget
import queue

class QWidgetDraw(QWidget):
    def __init__(self, parent=None):
        super(QWidgetDraw, self).__init__(parent)
        self.timer = QBasicTimer()
        self.timer.start(10, self)
        self.pixmap = QPixmap(self.width(), self.height()) # 创建与界面等大小的画布
        self.pixmap.fill(Qt.black) # 设置背景色为黑色
        self.queue = queue.Queue()
        
        # 变量定义
        self.index = 0 # 当前水平方向数据更新位置
        self.lastData = 0
    
    def timerEvent(self, event):
        if event.timerId() == self.timer.timerId():
            if self.queue.empty():
                self.drawWaveToPixmap(2048)
            else:
                self.drawWaveToPixmap(self.queue.get())
            self.update()
        else:
            print("skip unknown timerId event:", event.timerId)
            
    # 串口数据信号槽函数
    def slotSerialDataHandle(self, data):
        self.queue.put(data)
        
    def drawWaveToPixmap(self, data=None):
        if self.lastData == 0:
            self.lastData = data
            return
        pixPainter = QPainter()
        pixPainter.begin(self.pixmap)
        self.drawEcgWave(pixPainter, data) # 把接收到的数据绘制到pixmap上
        pixPainter.end()
        
    def paintEvent(self, event):
        painter = QPainter()
        painter.begin(self)
        painter.resetTransform() # 坐标系复位
        painter.drawPixmap(0, 0, self.pixmap)
        painter.resetTransform() # 坐标系复位
        painter.end()
        
    def drawEcgWave(self, painter, data):
        # 配置painter画笔参数
        pen = QPen()
        pen.setWidth(2)
        pen.setColor(QColor("#F40002"))
        painter.setPen(pen)
        
        # 擦除当前列
        painter.save() # 保持当前配置
        pen.setColor(Qt.black) # 修改painter画笔颜色配置
        painter.setPen(pen)
        pen.setWidth(8)
        painter.drawLine(self.index+4, 0, self.index+4, self.height())
        painter.restore() # 还原当前配置
        
        # 计算绘图参数
        height = self.height()

        # 更新绘图参数，水平方向循环绘图
        self.index = self.index + 2
        if self.index > self.width():
            self.index = 2

        lineStart = QPoint()
        lineEnd = QPoint()
        
        # 设置起始、结束点的x坐标
        lineStart.setX(self.index-2)
        lineEnd.setX(self.index)
        # 设置起始点的y坐标
        lineStart.setY(round(self.height()/2 - ((self.lastData - 2048)/600) * self.height()/2))
        # 设置结束点的y坐标
        lineEnd.setY(round(self.height()/2 - ((data - 2048)/600) * self.height()/2))
        self.lastData = data
        
        # 直线绘制
        painter.drawLine(lineStart, lineEnd)
                        
# step 3
class EcgWave(QWidget):
    def __init__(self, parent = None):
        super(QWidget, self).__init__()
        self.setWindowFlags (Qt.FramelessWindowHint) #隐藏标题栏
        self.setStyleSheet("background-color: black") 
        
        self.title = QLabel("ECG")
        self.title.setStyleSheet("color: white")
        self.waveWin = QWidgetDraw()
        self.layout = QVBoxLayout()
        
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.waveWin)
        self.layout.setStretch(0,1)
        self.layout.setStretch(1,5)
        self.setLayout(self.layout)

In [4]:
## 先执行
# 1. 串口数据接收类 SerialInterface_0v4
# 2. 串口数据解析类 EcgDecode_0v1
# 3. 界面显示类 QWidgetDraw_0v2

import math

from PyQt5.QtSerialPort import QSerialPort
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QSizePolicy, QLabel
from PyQt5.QtGui import QPainter, QColor, QPolygon, QPen, QPixmap
from PyQt5.QtCore import pyqtSignal, QObject, QTimer, Qt, QRectF, QPoint, Qt, QBasicTimer

app = QApplication([])

serial = SerialInterface('COM4')

wave = EcgWave()
wave.resize(600, 150)

serial.signal.connect(wave.waveWin.slotSerialDataHandle)

wave.show()

app.exec_()

ecgDecode init done
serial ready
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unkno

unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0

unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 8
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0

unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0

unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0

unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0

unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0
unknown id 0


0

# 心率测量


心率测量是从心电波形中获取心率的一种方法。这里面有各种各样的算法可以实现心率的计算，简单点描述就是测2个次心跳之间的时间。这里列出几种常用的方法：
* FFT傅里叶变换：傅里叶变换提供了频域和时域之间的桥梁，所谓时域就是正弦波，方波看到的波形，描述的是信号随着时间的一个变化；所谓频域指的是某一瞬间信号的组成，例如方波可以展开为无数谐波(方波频率的整数倍)的求和，把这些谐波画出来就是频率的波形了，水平方向是频率垂直方向是信号强度。通过对信号的傅里叶变化，可以找到幅度最大的基波频率，即心率.
* 小波变换：在FFT的基础上进行改进。[参考阅读](https://blog.csdn.net/nie15870449223/article/details/85097791)
* 阈值测算周期：根据波形上升沿的时间间隔计算周期。[参考阅读](https://www.cnblogs.com/whik/p/10631271.html)

实现上可用第三种方案，参考第三种方案：
1. 已知采样间隔，每秒500次
2. 已知信号范围0~4096，中值为2048，可设置阈值为2048+（4096-2048）/2
3. 计算相邻2次越过阈值时中间的采样点数。
4. 心率为每分钟的心跳次数，即每分钟越过阈值的次数，假定采样率每秒500次，每分钟为500*60次，假定阈值间隔为200，心率为：500*60/200=150BPM

# 任务

使用模拟器打开串口5并发送数据，然后波形显示程序打开串口4接收数据并显示

用Qt设计一个心电数据模拟器。
* 使用L6中的数据为ECG2的采集数据，对数据进行打包然后代替串口助手完成数据心电数据发送
* 心电数据每秒钟有500个数据采样点。请合理调整本篇代码中相关参数使波形显示正常